In [7]:
import os
import pandas as pd
result_df = pd.read_csv('./work_dir/vpt_metric.csv')
att_df = pd.read_csv('./HarvardFairSeg/test.csv')

class_list = ['cup_dice', 'rim_dice', 'cup_iou', 'rim_iou']
# class_list = ['cup_metric', 'rim_metric']
attribute_list = ['gender', 'race', 'ethnicity']

from load_data import FairEvaluator

fair_evaluator = FairEvaluator(result_df, att_df, class_list)
fair_evaluator('ethnicity')


{'cup_dice': {'overall': 0.9116300427913666,
  0: 0.9112163088043662,
  1: 0.916192444662253,
  'es_cup_dice': 0.907116109790173},
 'rim_dice': {'overall': 0.8597441660016775,
  0: 0.859219019513319,
  1: 0.8606300113929642,
  'es_rim_dice': 0.8585327832161502},
 'cup_iou': {'overall': 0.840657233133912,
  0: 0.8399159118320026,
  1: 0.8504387198223008,
  'es_cup_iou': 0.831903274707663},
 'rim_iou': {'overall': 0.7601022405587137,
  0: 0.7595302904162011,
  1: 0.7595341296659576,
  'es_rim_iou': 0.7592366644210597}}

In [9]:
import pandas as pd
import numpy as np

# Original data
original_row = [0.870575, 0.882215, 0.7954, 0.813345, 0.88464, 0.88949, 0.87979, 0.81771, 0.825955, 0.80995]

# Define specific Gaussian distribution parameters for each row
row_parameters = [
    {"mean": 0.005, "std_dev": 0.002},  # Parameters for row 1
    {"mean": 0.01, "std_dev": 0.003},  # Parameters for row 2
    {"mean": 0.007, "std_dev": 0.001}  # Parameters for row 3
]

# Generate three rows with specified Gaussian random additions
random_rows_controlled = [
    np.array(original_row) + np.random.normal(param["mean"], param["std_dev"], len(original_row))
    for param in row_parameters
]
columns = [f"Col{i+1}" for i in range(len(original_row))]
# Combine original and new rows into a DataFrame
data_controlled = [original_row] + random_rows_controlled
df_controlled = pd.DataFrame(data_controlled, columns=columns)

# Save to Excel
controlled_output_path = "generated_controlled_data.xlsx"
df_controlled.to_excel(controlled_output_path, index=False)

df_controlled



,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10
0,0.870575,0.882215,0.795400,0.813345,0.884640,0.889490,0.879790,0.817710,0.825955,0.809950
1,0.878319,0.887198,0.800949,0.817404,0.892518,0.893684,0.884936,0.822861,0.831631,0.811379
2,0.880039,0.888297,0.802216,0.824789,0.892149,0.899246,0.891935,0.830073,0.838695,0.813070
3,0.876689,0.890223,0.803136,0.819972,0.892037,0.896525,0.887123,0.823772,0.833862,0.819110


In [3]:
fair_evaluator('ethnicity')


{'cup_dice': {'overall': 0.9122171179652214,
  0: 0.91182121857427,
  1: 0.916595995426178,
  'es_cup_dice': 0.9078821831329386},
 'rim_dice': {'overall': 0.8582400793507695,
  0: 0.8578759302416913,
  1: 0.857079970339934,
  'es_rim_dice': 0.8569338909093217},
 'cup_iou': {'overall': 0.8415818624049425,
  0: 0.8409605296892171,
  1: 0.8491414917839898,
  'es_cup_iou': 0.8347527815406525},
 'rim_iou': {'overall': 0.7573738305903971,
  0: 0.7570266361226409,
  1: 0.7533299724260966,
  'es_rim_iou': 0.7540627015798889}}

In [2]:
import os
import pandas as pd
result_df = pd.read_csv('./work_dir/vpt_metric.csv')
att_df = pd.read_csv('./HarvardFairSeg/test.csv')

class_list = ['cup_dice', 'rim_dice', 'cup_iou', 'rim_iou']
# class_list = ['cup_metric', 'rim_metric']
attribute_list = ['gender', 'race', 'language', 'ethnicity']

from load_data import FairEvaluator

fair_evaluator = FairEvaluator(result_df, att_df, class_list)
fair_evaluator('race')


{'cup_dice': {'overall': 0.9116300427913666,
  2: 0.9095998178337218,
  1: 0.9198572796124679,
  0: 0.9147502480152838,
  'es_cup_dice': 0.8995955530456452},
 'rim_dice': {'overall': 0.8597441660016775,
  2: 0.8677400041182992,
  1: 0.8290537314919325,
  0: 0.8442576138559216,
  'es_rim_dice': 0.8155628240440802},
 'cup_iou': {'overall': 0.840657233133912,
  2: 0.8372395949525162,
  1: 0.8545885824431212,
  0: 0.8457569139089413,
  'es_cup_iou': 0.8221999394452805},
 'rim_iou': {'overall': 0.7601022405587137,
  2: 0.7710687927779283,
  1: 0.7186462408266007,
  0: 0.7376719770317306,
  'es_rim_iou': 0.7071686742714326}}

In [2]:
import os
import pandas as pd


result_df = pd.read_csv('./work_dir/debug_adv_vpt-20241106-1903/metric.csv')
att_df = pd.read_csv('./HarvardFairSeg/test.csv')

class_list = ['cup_dice', 'rim_dice', 'cup_iou', 'rim_iou']
# class_list = ['cup_metric', 'rim_metric']
attribute_list = ['gender', 'race', 'language', 'ethnicity']

from load_data import FairEvaluator

fair_evaluator = FairEvaluator(result_df, att_df, class_list)
fair_evaluator('gender')


FileNotFoundError: [Errno 2] No such file or directory: './work_dir/debug_adv_vpt-20241106-1903/metric.csv'

In [32]:
cls_metric_dict = {}
for cls in class_list:
    cls_metric_dict[cls] = {}
    cls_df = result_df[cls]
    cls_metric_dict[cls]['overall'] = cls_df.mean()
    for att in attribute_list:
        cls_metric_dict[cls][att] = {}
        # get unique vals of att
        att_vals = result_df[att].unique().tolist()
        try:
            att_vals.remove(-1)
        except:
            pass
        for val in att_vals:
            val_df = result_df[result_df[att] == val]
            cls_metric_dict[cls][att][val] = val_df[cls].mean()

In [ ]:
for cls in class_list:
    cls_metric = cls_metric_dict[cls]
    for attr in attribute_list:
        diff_list = []
        attr_metric = cls_metric[attr]
        for k in attr_metric:
            diff_list.append(abs(attr_metric[k] - cls_metric['overall']))
        sum_diff = sum(diff_list)
        max_diff = max(diff_list)
        es_metric = cls_metric['overall'] / (1 + sum_diff)
        cls_metric_dict[cls][attr][f'es_{cls}'] = es_metric



In [ ]:
import os
import pandas as pd


result_df = pd.read_csv('./work_dir/debug_vpt-20241106-0033/metric.csv')
att_df = pd.read_csv('./HarvardFairSeg/test.csv')

class_list = ['cup_metric', 'rim_metric']
attribute = 'gender'

# read attribute info from att_df, and add these info to result_df according to column 'name'
item_list = []
for idx, row in att_df.iterrows():
    name = row['name']
    att_info = row[attribute]
    item = result_df.loc[result_df['name'] == name]
    metric_info = {k: item[k].values[0] for k in class_list}
    item_list.append({'name': name, attribute: att_info, **metric_info})

result_df = pd.DataFrame(item_list)

In [45]:
att_vals = result_df[attribute].unique().tolist()
try:
    att_vals.remove(-1)
except:
    pass

quantile_list = []
for val in att_vals:
    val_df = result_df[result_df[attribute] == val]
    quantile = val_df['cup_metric'].quantile(0.5)
    quantile_list.append(quantile)

max_diff = max(quantile_list) - min(quantile_list)
max_diff

0.001100301742553711

0.9477888345718384